In [18]:
import pandas as pd

%run DataSource.ipynb
%run Metric.ipynb
%run Market.ipynb

report_columns = ['ma_short','ma_mid','total_profit',
                  'win_rate','loss_times','max_retrace',
                  'deal_count','win_count',
                  'period_score','retrace_score']

Buy_type='Buy'
Sell_type='Sell'
deal_open='open'
deal_closed='done'

class Account:
    init_money = 0
    start_date = ''
    account_money = 0
    
    floating_profits = []
    operation_detail= pd.DataFrame(columns=['date','profit_rate','account_money'])
    
    def __init__(self,init_money,start_date,end_date,short,mid):
        self.init_money = init_money
        self.account_money = init_money
        
        self.start_date = start_date
        self.end_date = end_date
        
        self.short = short
        self.mid = mid
        
        self.floating_profits = []
        
        self.i = 0
        self.operation_detail = pd.DataFrame(columns=['date','profit_rate','account_money'])
        self.operation_detail.loc[self.i] = [start_date,0,self.init_money]
        self.i +=1
        
        self.deal_df = pd.DataFrame(columns = ['symbol', 'date', 'price', 'shares', 
                                               'type', 'space','status'])
        self.position = {}
        self.metric_d_k = {}
        self.increase_per = {}
        self.asset = pd.DataFrame(columns = ['date','assets','space_per'])
        self.market = Market(self.short, self.mid)
        
        
    def get_init_money(self):
        return self.init_money
    
    def get_all_cash(self):
        return self.account_money
    
    def buy_stock(self,symbol,date,price,buy_shares):
        space_value = 100*(price*buy_shares)/self.account_money
        self.deal_df.loc[self.deal_df.shape[0] + 1] = [symbol,date,price,buy_shares,Buy_type, space_value,'open']
        
        if(symbol in self.position.keys()):
            self.position[symbol] = self.position[symbol] + buy_shares 
        else:
            self.position[symbol] = buy_shares
         
        self.account_money -= buy_shares*price
        self.metric_d_k[symbol]=Metric(symbol, level_day, self.short, self.mid)
        
        market_status = self.market.get_market_status(date)
        print(date+' buy '+symbol +' price:'+str(price)+ ' '+str(buy_shares) +' position:' + str(self.position) +' '+market_status+' space:'+str(self.get_space_per(date)))
    
    def sell_stock(self,symbol,date,price,sell_shares):
        if(symbol in self.position.keys()):
            self.position[symbol] = self.position[symbol] - sell_shares 
        else:
            print('Warning!!, dirty data ' +symbol+' '+ date+ ' has no shares')
        
        self.deal_df.loc[self.deal_df.shape[0] + 1] = [symbol,date,price,sell_shares,Sell_type, 0, deal_open]
        
        if(self.position[symbol]<=0):
            del(self.position[symbol])
            del(self.metric_d_k[symbol])
            del(self.increase_per[symbol])
            self.deal_df.loc[(self.deal_df['symbol']==symbol) & (self.deal_df['status']==deal_open),'status']=deal_closed
    
        self.account_money += sell_shares * price
        print(date+' sell '+symbol +' price:'+str(price)+ ' '+str(sell_shares) +' position:' + str(self.position))
        
        
    def get_shares(self,symbol):
        return self.position[symbol]

    def can_open_new_stock(self):
        if(len(self.position)>4):
            return False
        else:
            return True
        
    def has_shares(self):
        for item in self.position.values():
            if(item>0):
                return True

        return False
    
    def can_sell_stock(self):
        return self.has_shares()
    
    
    def get_latest_deal(self,symbol):
        deals = self.deal_df[(self.deal_df['symbol']==symbol) & 
                                   (self.deal_df['type']==Buy_type)]
        
        return deals[deals.shape[0]-1:]
        
    
    def get_last_time_buy_price(self,symbol):
        latest_deal = self.get_latest_deal(symbol)
        
        return latest_deal['price'].values[0]
    
    def get_last_time_buy_date(self,symbol):
        latest_deal = self.get_latest_deal(symbol)
        
        return latest_deal['date'].values[0]
    
    def get_all_position_symbols(self):
        return self.position.keys()
    
    def get_asset_df(self):
        return self.asset
  
    def get_today_stock_asset(self, cur_date):
        all_symbols = self.position.keys()
        
        stock_asset = 0
        if(len(all_symbols)>0):
            for symbol in all_symbols:
                d_k_metric = self.metric_d_k[symbol]
                
                latest_price = d_k_metric.get_latest_price(cur_date)
                stock_asset += latest_price*self.position[symbol]
            
        return stock_asset
    
      
    def get_space_per(self,cur_date):
        today_stock_asset = self.get_today_stock_asset(cur_date)
        today_cash = self.account_money
        space_rate = round(100*(today_stock_asset/(today_stock_asset+today_cash)),2)
    
        return space_rate
        

    def daily_audit(self,cur_date):
        for symbol in self.position.keys():
            d_k_metric = self.metric_d_k[symbol]
            latest_price = d_k_metric.get_latest_price(cur_date)
            
            increase_rate_per = 100*(latest_price-self.get_last_time_buy_price(symbol))/self.get_last_time_buy_price(symbol)
            
            if(symbol in self.increase_per.keys()):
                old_per = self.increase_per[symbol]
                if(increase_rate_per>old_per):
                    self.increase_per[symbol] = increase_rate_per
            else:
                self.increase_per[symbol] = increase_rate_per
        
        space_rate = self.get_space_per(cur_date)
        
        today_stock_asset = self.get_today_stock_asset(cur_date)
        today_cash = self.account_money

        self.asset.loc[self.asset.shape[0]+1] = [cur_date,today_stock_asset+today_cash,space_rate]
        

        